# Assignment 2 

## Kun Huang, Yizhou Jin

In this assignment, we try some features in **TensorFlow** and **gym**. 

First we import all the libraries necessary. 

In [2]:
import gym
import numpy as np
import random
import tensorflow as tf
from IPython.display import clear_output
import time
import warnings

warnings.filterwarnings('ignore')

In [3]:
#Set environment in gym
env = gym.make('FrozenLake-v0')

## Codes 1

The idea mainly comes from [Simple Reinforcement Learning with Tensorflow Part 0: Q-Learning with Tables and Neural Networks](https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-0-q-learning-with-tables-and-neural-networks-d195264329d0). It uses the idea of neural networks, that is, it uses a one-layer network which takes the state encoded in a vector (1x16,i.e. `inputs1`), and produces a vector of 4 Q-values (i.e. `Qout` and `nextQ`). 

In [4]:
tf.reset_default_graph()

#These lines establish the feed-forward part of the network used to choose actions
inputs1 = tf.placeholder(shape=[1,16],dtype=tf.float32)
W = tf.Variable(tf.random_uniform([16,4],0,0.1))
Qout = tf.matmul(inputs1,W)
predict = tf.argmax(Qout,1)

# loss, L2
nextQ = tf.placeholder(shape=[1,4],dtype=tf.float32)
loss = tf.reduce_sum(tf.square(nextQ - Qout))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(loss)

Next, we run the codes rewritten in tensorflow.

In [9]:
# set a initializer for tensor
init = tf.global_variables_initializer()

# Set learning parameters
gamma = .7
episodes = 5000
alpha = 0.2

total = 0

with tf.Session() as sess:
    sess.run(init)
    for i in range(episodes):
        # Reset environment and retrieve the first new state
        state = env.reset()
        done = False
        
        epsilon = 1./(i+1)
        # First act
        action,allQ = sess.run([predict,Qout], feed_dict={inputs1:np.identity(16)[state:state + 1]})
        targetQ = allQ
        
        #The Q-Network
        while not done:
            #Choose an action
            if epsilon < random.uniform(0,1):
                action[0] = env.action_space.sample()
            else:
                action[0] = int(np.argmax(targetQ))
            #Get new state and reward from env
            new_state, reward, done, info = env.step(action[0])
            
            #Obtain the Q' values by feeding the new state through our network
            Q1 = sess.run(Qout, feed_dict={inputs1:np.identity(16)[new_state:new_state + 1]})
            #np.max(Q[new_state])
            maxQ1 = np.max(Q1)
            # formula
            targetQ[0,action[0]] += alpha * (reward + gamma*maxQ1 - targetQ[0, action[0]])
            #Train our network using target and predicted Q values
            _,W1 = sess.run([updateModel,W],feed_dict={inputs1:np.identity(16)[state:state+1],nextQ:targetQ})
            total += reward
            state = new_state
            
print ("Average rewards: " + str(total/episodes))
print("Learned Q-Table")
print(W1)


Average rewards: 0.0168
Learned Q-Table
[[0.00190608 0.00263399 0.00228829 0.00212742]
 [0.00416666 0.0054995  0.00342879 0.00216182]
 [0.00191839 0.00268508 0.00231669 0.0021076 ]
 [0.00697597 0.00402402 0.00431644 0.0022966 ]
 [0.00196856 0.0057706  0.00229834 0.00267015]
 [0.05660341 0.06357435 0.07471418 0.0885246 ]
 [0.00304796 0.00552304 0.00388466 0.00429829]
 [0.02196448 0.04746931 0.00585063 0.09813278]
 [0.00227244 0.00356025 0.00396797 0.00239674]
 [0.00626919 0.00451156 0.0026764  0.00291943]
 [0.00394859 0.00419651 0.00511321 0.004895  ]
 [0.05528345 0.00099499 0.0919311  0.0266538 ]
 [0.07350774 0.09006526 0.08291616 0.08375569]
 [0.00556652 0.0056868  0.00406212 0.00384539]
 [0.00573694 0.05363594 0.04415478 0.00960633]
 [0.03766136 0.00419477 0.09015631 0.0393252 ]]


## Code2

Similarly, we rewrite the sample code into tensorflow.

In [10]:
env.reset()
episodes = 10
max_steps = 100

with tf.Session() as sess:
    sess.run(init)
    for ep in range(episodes):
        state = env.reset()
        step = 0
        done = False
        clear_output()
        print("*****************")
        print("Episodes", ep)
        print("*****************")
        for step in range(max_steps):
            action,allQ = sess.run([predict,Qout], feed_dict={inputs1:np.identity(16)[state:state + 1]})
            action[0] = np.argmax(allQ)
            new_state, reward, done, info = env.step(action[0])
            if done:
                env.render()
                print("*****************")
                print("Steps", step)
                print("*****************")
                time.sleep(2)
                break
        state = new_state
env.close()


*****************
Episodes 9
*****************
  (Left)
SFFF
FHFH
FFFH
HFFG
*****************
Steps 13
*****************
